In [12]:
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")
using MLJ
using MLDataPattern
using DataFrames
using XGBoost
using MLJXGBoostInterface
using Plots
using MultivariateStats
using MLJBase
using Printf
using UMAP
using ThreadSafeDicts
using MLJ
using Printf
using CategoricalArrays
using BioAlignments

In [16]:
a= norm_grant
b = [0.0 0.4985 0.491 0.4958 0.5012 0.4979 0.4886 0.4829 0.4933 0.4937 0.4915 0.4955 0.4917 0.485 0.4924 0.4919 0.4879 0.4866 0.492 0.4892; 0.4985 0.0 0.4945 0.4973 0.5031 0.489 0.4939 0.4938 0.4928 0.4852 0.4932 0.49 0.4834 0.4965 0.4947 0.4828 0.4832 0.4887 0.4885 0.4919; 0.491 0.4945 0.0 0.4934 0.4966 0.4935 0.496 0.4951 0.4877 0.5013 0.4907 0.4871 0.4895 0.49 0.4852 0.4977 0.4933 0.4928 0.4974 0.4924; 0.4958 0.4973 0.4934 0.0 0.5038 0.5121 0.4908 0.4975 0.4943 0.4935 0.4865 0.4925 0.4873 0.4874 0.4984 0.4931 0.4939 0.4904 0.4896 0.497; 0.5012 0.5031 0.4966 0.5038 0.0 0.4887 0.492 0.4979 0.4937 0.4897 0.4965 0.4927 0.4923 0.5016 0.4908 0.4803 0.4951 0.4922 0.4986 0.4946; 0.4979 0.489 0.4935 0.5121 0.4887 0.0 0.4925 0.4944 0.4936 0.502 0.4932 0.4946 0.4956 0.4987 0.4931 0.4868 0.4994 0.4949 0.4957 0.4805; 0.4886 0.4939 0.496 0.4908 0.492 0.4925 0.0 0.4887 0.4991 0.4953 0.4939 0.4927 0.4991 0.4892 0.4974 0.4927 0.4985 0.4878 0.4962 0.4876; 0.4829 0.4938 0.4951 0.4975 0.4979 0.4944 0.4887 0.0 0.4956 0.4916 0.4892 0.497 0.501 0.4911 0.5045 0.4896 0.4898 0.4907 0.4929 0.4915; 0.4933 0.4928 0.4877 0.4943 0.4937 0.4936 0.4991 0.4956 0.0 0.5002 0.4988 0.495 0.4948 0.4983 0.4907 0.4848 0.4912 0.4929 0.4861 0.4915; 0.4937 0.4852 0.5013 0.4935 0.4897 0.502 0.4953 0.4916 0.5002 0.0 0.4984 0.4852 0.4858 0.4973 0.4911 0.501 0.4912 0.4843 0.4861 0.4887; 0.4915 0.4932 0.4907 0.4865 0.4965 0.4932 0.4939 0.4892 0.4988 0.4984 0.0 0.4868 0.4998 0.4865 0.4963 0.4982 0.495 0.4889 0.4929 0.4937; 0.4955 0.49 0.4871 0.4925 0.4927 0.4946 0.4927 0.497 0.495 0.4852 0.4868 0.0 0.4942 0.4927 0.4861 0.4904 0.4972 0.4921 0.4901 0.4905; 0.4917 0.4834 0.4895 0.4873 0.4923 0.4956 0.4991 0.501 0.4948 0.4858 0.4998 0.4942 0.0 0.4939 0.4941 0.4922 0.4826 0.4785 0.4893 0.4839; 0.485 0.4965 0.49 0.4874 0.5016 0.4987 0.4892 0.4911 0.4983 0.4973 0.4865 0.4927 0.4939 0.0 0.4942 0.4965 0.4905 0.4958 0.496 0.4892; 0.4924 0.4947 0.4852 0.4984 0.4908 0.4931 0.4974 0.5045 0.4907 0.4911 0.4963 0.4861 0.4941 0.4942 0.0 0.4977 0.4861 0.4926 0.4934 0.493; 0.4919 0.4828 0.4977 0.4931 0.4803 0.4868 0.4927 0.4896 0.4848 0.501 0.4982 0.4904 0.4922 0.4965 0.4977 0.0 0.4938 0.4843 0.4867 0.4879; 0.4879 0.4832 0.4933 0.4939 0.4951 0.4994 0.4985 0.4898 0.4912 0.4912 0.495 0.4972 0.4826 0.4905 0.4861 0.4938 0.0 0.4901 0.4929 0.4909; 0.4866 0.4887 0.4928 0.4904 0.4922 0.4949 0.4878 0.4907 0.4929 0.4843 0.4889 0.4921 0.4785 0.4958 0.4926 0.4843 0.4901 0.0 0.4816 0.4934; 0.492 0.4885 0.4974 0.4896 0.4986 0.4957 0.4962 0.4929 0.4861 0.4861 0.4929 0.4901 0.4893 0.496 0.4934 0.4867 0.4929 0.4816 0.0 0.4854; 0.4892 0.4919 0.4924 0.497 0.4946 0.4805 0.4876 0.4915 0.4915 0.4887 0.4937 0.4905 0.4839 0.4892 0.493 0.4879 0.4909 0.4934 0.4854 0.0]
c = hcat([[0.5 for i in 1:20] for i in 1:20]...)

heatmap(a, title="Target Grantham distances", clims=(0.0,1.0), dpi = 300)
savefig("../thesis/Fig/granttarget.png")

heatmap(b, title="Achieved Grantham distances", clims=(0.0,1.0), dpi = 300)
savefig("../thesis/Fig/grantachieved.png")

heatmap(c, title="Starting similarities", clims=(0.0,1.0), dpi = 300)
savefig("../thesis/Fig/random.png")

"/home/mfat/Master-Thesis/thesis/Fig/random.png"

In [14]:
AA_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

simmat = copy(GRANTHAM1974)
simmatblo = copy(BLOSUM62)

grantham = [[simmat[i, j] for i in AA_list] for j in AA_list]
grantham = hcat(grantham...)

blosum = [[simmatblo[i, j] for i in AA_list] for j in AA_list]
blosum = hcat(blosum...)

function norm_mat(matrix, zero=false)
    if zero == true
        min_value = 0
    else 
        min_value = minimum(matrix)
    end
    
    max_value = maximum(matrix)

    normalized_matrix = (matrix .- min_value) ./ (max_value - min_value)
    return normalized_matrix
end

norm_blosum = broadcast(abs, norm_mat(blosum) .-1)
setindex!.(Ref(norm_blosum), 0.0, 1:20, 1:20)
norm_grant = norm_mat(grantham, true)

20×20 Matrix{Float64}:
 0.0       0.906977  0.586047  0.497674  …  0.297674   0.688372  0.52093
 0.906977  0.0       0.716279  0.790698     0.893023   1.0       0.902326
 0.586047  0.716279  0.0       0.209302     0.706977   0.84186   0.744186
 0.497674  0.790698  0.209302  0.0          0.562791   0.706977  0.567442
 0.525581  0.953488  0.823256  0.651163     0.232558   0.186047  0.102326
 0.27907   0.739535  0.437209  0.455814  …  0.506977   0.855814  0.683721
 0.4       0.809302  0.376744  0.186047     0.390698   0.534884  0.386047
 0.437209  0.92093   0.781395  0.623256     0.134884   0.283721  0.153488
 0.493023  0.939535  0.469767  0.260465     0.451163   0.511628  0.395349
 0.446512  0.92093   0.8       0.64186      0.148837   0.283721  0.167442
 0.390698  0.911628  0.744186  0.586047  …  0.0976744  0.311628  0.167442
 0.516279  0.646512  0.106977  0.195349     0.618605   0.809302  0.665116
 0.125581  0.786047  0.502326  0.432558     0.316279   0.683721  0.511628
 0.423256  0.716

In [15]:
norm_blosum

20×20 Matrix{Float64}:
 0.0       0.733333  0.866667  0.8       …  0.733333  0.933333  0.866667
 0.733333  0.0       0.933333  1.0          0.8       0.866667  0.866667
 0.866667  0.933333  0.0       0.6          0.933333  1.0       0.933333
 0.8       1.0       0.6       0.0          0.866667  0.933333  0.866667
 0.866667  0.866667  0.933333  0.933333     0.8       0.666667  0.533333
 0.733333  0.933333  0.8       0.866667  …  0.933333  0.866667  0.933333
 0.866667  0.933333  0.8       0.733333     0.933333  0.866667  0.6
 0.8       0.8       0.933333  0.933333     0.533333  0.933333  0.8
 0.8       0.933333  0.8       0.666667     0.866667  0.933333  0.866667
 0.8       0.8       1.0       0.933333     0.666667  0.866667  0.8
 0.8       0.8       0.933333  0.866667  …  0.666667  0.8       0.8
 0.866667  0.933333  0.666667  0.733333     0.933333  1.0       0.866667
 0.8       0.933333  0.8       0.8          0.866667  1.0       0.933333
 0.8       0.933333  0.733333  0.6          0.86